In [1]:
import torch

print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
if torch.cuda.is_available():
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")


CUDA available: True
CUDA version: 12.1
GPU Device: NVIDIA GeForce RTX 4090


In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

huggingface_api_key = os.getenv("HUGGINGFACE_API_KEY")
wandb_api_key = os.getenv("WANDB_API_KEY")

if not huggingface_api_key:
    raise ValueError("HUGGINGFACE_API_KEY not set")

if not wandb_api_key:
    raise ValueError("WANDB_API_KEY not set")

In [3]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging
)

from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model
)

import wandb
import pandas as pd
from datasets import Dataset
from trl import SFTTrainer, setup_chat_format

E:\PyCharm\PycharmProjects\repurposed-llm-phishing-classifier\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]


In [4]:
from huggingface_hub import login

login(token=huggingface_api_key)

In [5]:
wandb.login(key=wandb_api_key)

run = wandb.init(
    project="repurposed-llm-phishing-classifier",
    job_type="train",
    anonymous="allow"
)

wandb: Currently logged in as: morganj-lee01 (morganj-lee01-synpulse8). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\morga\_netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [6]:
base_model_path = "../models/llama_models/llama-3.2-3B"
new_model_path = "../models/tuned_models/llama-3.2-3B-phishing-classifier"
dataset_path = "../processed_data/train.csv"

In [7]:
train_df = pd.read_csv(dataset_path)

train_df.head()

,system,user,assistant
0,You are a classification system designed to ca...,Message for review: write me back please year ...,True
1,You are a classification system designed to ca...,Message for review: I just picked up Razor SDK...,False
2,You are a classification system designed to ca...,"Message for review: vacation goodmorning , i w...",False
3,You are a classification system designed to ca...,"Message for review: On Thu, Aug 08, 2002 at 11...",False
4,You are a classification system designed to ca...,Message for review: wellheads shoreline has se...,False


In [8]:
model_config = {
    "torch_dtype": torch.bfloat16,
    "attn_implementation": "flash_attention_2",
    "device_map": "auto"
}

In [9]:
# WARNING: flash_attention_2 required pip install flash-attn which needs C++ builds tools
# It also takes absolutely forever to compile because it's compiling CUDA kernels

model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    device_map=model_config["device_map"],
    torch_dtype=model_config["torch_dtype"],
    attn_implementation=model_config["attn_implementation"]
)

ImportError: FlashAttention2 has been toggled on, but it cannot be used due to the following error: the package flash_attn seems to be not installed. Please refer to the documentation of https://huggingface.co/docs/transformers/perf_infer_gpu_one#flashattention-2 to install Flash Attention 2.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model_path, trust_remote_code=True)

In [ ]:
from datasets import Dataset
import pandas as pd

df = pd.read_csv("your_phishing_data.csv")

dataset = Dataset.from_pandas(df)

def format_chat_template(row):

    row_json = [
        {"role": "system", "content": row["system"]},
        {"role": "user", "content": row["user"]},
        {"role": "assistant", "content": row["assistant"]}
    ]

    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

formatted_dataset = dataset.map(
    format_chat_template,
    num_proc=4,
)